In [1]:
import os
from pyspark.sql import SparkSession
import pyspark
from pyspark import SparkConf, SparkContext

In [20]:
spark.stop()  

In [2]:
AZURE_STORAGE_JAR = "/home/azureuser/pyspark_env/src/real_estate/real_estate/notebooks/azure-storage-8.6.6.jar"
HADOOP_AZURE_JAR = "/home/azureuser/pyspark_env/src/real_estate/real_estate/notebooks/hadoop-azure-2.7.1.jar"
AZURE_STORAGE_BLOB = "/home/azureuser/pyspark_env/src/real_estate/real_estate/notebooks/azure-storage-blob-12.17.1.jar"
JETTY_UTILS = "/home/azureuser/pyspark_env/src/real_estate/real_estate/notebooks//jetty-util-7.0.0.pre5.jar"

In [3]:
spark = SparkSession.builder.appName("using_wg")\
        .config("spark.driver.memory","4g")\
        .config("spark.jars",f"{AZURE_STORAGE_JAR},{HADOOP_AZURE_JAR},{AZURE_STORAGE_BLOB},{JETTY_UTILS}")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.executor.cores","2").getOrCreate()

22/07/09 23:34:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
storage_account_name = "realestatedb"
storage_account_key = "vhD06bMo8z+dZ6OuV6mNDnj7nRqz5l4ybcrJyYGMG0Nz0O/DsoPloxVZBcsY9dXgxliw+yVcI0Fe+AStl1tEmA=="
container = "container1"

In [5]:
SparkSession.getActiveSession()

In [6]:
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)
#spark.conf.set("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem")
sc._jsc.hadoopConfiguration().set("fs.azure","org.apache.hadoop.fs.azure.NativeAzureFileSystem")
sc._jsc.hadoopConfiguration().set("spark.hadoop.fs.wasbs.impl","org.apache.hadoop.fs.azure.NativeAzureFileSystem")
sc._jsc.hadoopConfiguration().set("fs.wasbs.impl","org.apache.hadoop.fs.azure.NativeAzureFileSystem")
sc._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.wasbs.impl", "org.apache.hadoop.fs.azure.Wasbs")
sc._jsc.hadoopConfiguration().set("spark.hadoop.fs.adl.impl","org.apache.hadoop.fs.adl.AdlFileSystem")

In [7]:
sc._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.driver.memory', '4g'),
 ('spark.driver.port', '42131'),
 ('spark.app.id', 'local-1657409665315'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.cores', '2'),
 ('spark.app.initial.jar.urls',
  'spark://bdproject.interna

In [10]:
file = spark.read.csv(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/marks.csv", header=True, inferSchema=True)

In [8]:
data = spark.range(0,5)
data.write.format("delta").save("tmp/delta-table")

Py4JJavaError: An error occurred while calling o74.save.
: java.lang.ClassNotFoundException: 
Failed to find data source: delta. Please find packages at
https://spark.apache.org/third-party-projects.html
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedToFindDataSourceError(QueryExecutionErrors.scala:574)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:675)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:725)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:864)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:256)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:239)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	